Now that you've learned about random forests and decision trees let's do an exercise in accuracy. You know that random forests are basically a collection of decision trees. But how do the accuracies of the two models compare?

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

Hopefully out of this you'll see the power of random forests, but also their potential costs. Remember, in the real world you won't necessarily be dealing with thousands of rows. It could be millions, billions, or even more.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
# We import the data from csv file:
df = pd.read_csv('think capstone1_slim.csv' ,encoding ='latin1')


In [48]:
# Get a feel for the data:
df.head()

,YYYY_MM,IdProduct,Product_Name,Manufacturer_Name,Product Classification,IdLocation,Drugstore_Name,Total_Sales,No_of_transactions,No_of_medical_prescriptions,Units_Sold,Net_Profit,row_num_sales,row_num_profit
0,2015-1,20177,Nut Seringa Ject 10 Ml Cu Ac,MINUT,_DispozitivMedicalMultiplu,3,Stad,45.73,15,0,86.0,18.07,2,3
1,2015-1,2130,Panthen Unguent 50 Mg /G X 100 G,BAYER SCHERING,Cosmetice,3,Stad,265.55,4,0,5.0,46.66,1,1
2,2015-1,18,Oderma Abc Derm Gel Spumant X 200 Ml,BIODERMA,Cosmetice,3,Stad,37.42,1,0,1.0,9.45,14,9
3,2015-1,20697,Ltene Tratament Pt Unghii,FOLTENE,Cosmetice,3,Stad,48.90,1,0,1.0,4.38,8,33
4,2015-1,25030,Nsiprod Tensiometru Pentru Brat,PANSIPROD,Dispozitive Medicale,3,Stad,408.16,2,0,2.0,96.13,1,2


In [37]:
# What kind of data is in the dataframe?
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 14 columns):
YYYY_MM                        1686 non-null object
IdProduct                      1686 non-null int64
Product_Name                   1686 non-null object
Manufacturer_Name              1686 non-null object
Product Classification         1686 non-null object
IdLocation                     1686 non-null int64
Drugstore_Name                 1686 non-null object
Total_Sales                    1686 non-null float64
No_of_transactions             1686 non-null int64
No_of_medical_prescriptions    1686 non-null int64
Units_Sold                     1686 non-null float64
Net_Profit                     1686 non-null float64
row_num_sales                  1686 non-null int64
row_num_profit                 1686 non-null int64
dtypes: float64(3), int64(6), object(5)
memory usage: 184.5+ KB
None


In [49]:
categorical = df.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print('Column name:',i ,' \t\t/ Unique values:', column.nunique())

Column name: YYYY_MM  		/ Unique values: 3
Column name: Product_Name  		/ Unique values: 488
Column name: Manufacturer_Name  		/ Unique values: 132
Column name: Product Classification  		/ Unique values: 12
Column name: Drugstore_Name  		/ Unique values: 5


In [50]:
pd.get_dummies(df)

,IdProduct,IdLocation,Total_Sales,No_of_transactions,No_of_medical_prescriptions,Units_Sold,Net_Profit,row_num_sales,row_num_profit,YYYY_MM_2015-1,...,Product Classification_Parafarm.,Product Classification_RX,Product Classification_RX - 120,Product Classification_Suplimente,Product Classification__DispozitivMedicalMultiplu,Drugstore_Name_Bica,Drugstore_Name_Buhu,Drugstore_Name_Ener,Drugstore_Name_Repu,Drugstore_Name_Stad
0,20177,3,45.73,15,0,86.00,18.07,2,3,1,...,0,0,0,0,1,0,0,0,0,1
1,2130,3,265.55,4,0,5.00,46.66,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,18,3,37.42,1,0,1.00,9.45,14,9,1,...,0,0,0,0,0,0,0,0,0,1
3,20697,3,48.90,1,0,1.00,4.38,8,33,1,...,0,0,0,0,0,0,0,0,0,1
4,25030,3,408.16,2,0,2.00,96.13,1,2,1,...,0,0,0,0,0,0,0,0,0,1
5,22321,3,41.94,1,0,1.00,13.86,19,10,1,...,0,0,0,0,0,0,0,0,0,1
6,23861,3,169.68,1,1,2.00,8.84,6,18,1,...,0,0,0,0,0,0,0,0,0,1
7,22102,3,378.08,2,0,2.00,-3.00,2,42,1,...,0,0,0,0,0,0,0,0,0,1
8,13427,3,736.71,86,0,68.85,154.69,6,8,1,...,0,0,0,0,0,0,0,0,0,1
9,1957,3,129.33,3,0,3.00,34.51,7,5,1,...,1,0,0,0,0,0,0,0,0,1


### Random Forest

In [40]:
# This is the model we'll be using.
from sklearn import tree

In [41]:
from sklearn.model_selection import train_test_split

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(1348, 648) (1348,)
(338, 648) (338,)


In [42]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

import time
start_time = time.time()

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

print("--- %s seconds ---" % (time.time() - start_time))

--- 16.275256872177124 seconds ---


### Make Predictions on the Test Set
We compare the predictions to the known answers. When performing regression, we need to make sure to use the absolute error because we expect some of our answers to be low and some to be high. We are interested in how far away our average prediction is from the actual value so we take the absolute value

In [43]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2 ), 'Net_Profit')

Mean Absolute Error: 26.78 Net_Profit


In [44]:
# Now let's compare some of our predicted values with the actual values and see how accurate we were:
df=pd.DataFrame({'Actual':y_test, 'Predicted':predictions})  
df  

,Actual,Predicted
1566,555.20,518.76043
1569,203.11,205.29431
799,765.17,657.44005
1040,744.33,626.12941
307,79.39,87.57340
975,1485.84,1436.13534
996,32.04,30.24034
544,276.66,298.50674
460,95.54,107.25356
1155,10.74,10.79679


### Determine Performance Metrics
To put our predictions in perspective, we can calculate an accuracy using the mean average percentage error subtracted from 100 %.

In [45]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 106.5 %.


In [46]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

import time
start_time = time.time()

rfc = ensemble.RandomForestRegressor()
X = df.drop('Net_Profit', axis=1)
Y = df['Net_Profit']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

print(cross_val_score(rfc, X, Y, cv=10))


print("--- %s seconds ---" % (time.time() - start_time))

ValueError: labels ['Net_Profit'] not contained in axis

# Decision Tree:

In [9]:
# This is the model we'll be using.
from sklearn import tree

In [11]:
from sklearn.model_selection import train_test_split

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(1348, 648) (1348,)
(338, 648) (338,)


In [31]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

import time
start_time = time.time()

regressor = DecisionTreeRegressor(max_depth=5)
regressor.fit( X_train , y_train )

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.021056652069091797 seconds ---


### To make predictions on the test set, use the predict method:

In [15]:
y_pred = regressor.predict(X_test) 


In [17]:
# Now let's compare some of our predicted values with the actual values and see how accurate we were:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
df  

,Actual,Predicted
1566,555.20,542.682439
1569,203.11,285.064198
799,765.17,542.682439
1040,744.33,486.316667
307,79.39,64.274427
975,1485.84,974.374706
996,32.04,6.374043
544,276.66,285.064198
460,95.54,95.023429
1155,10.74,6.529397


### Evaluating the Algorithm
To evaluate performance of the regression algorithm, the commonly used metrics are 
- mean absolute error, 
- mean squared error, and 
- root mean squared error. 

In [18]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Mean Absolute Error: 55.83176294220861
Mean Squared Error: 9751.325866459145
Root Mean Squared Error: 98.74880184822064


In [29]:
# Calculate the absolute errors
errors = abs(y_pred - y_test)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 239.2 %.


### Overfitting ...?